In [13]:
import sentencepiece as spm

model = 'data/sp/subword_tokenizer_ko.model'
model2 = 'data/sp/char_tokenizer_ko.model'
input = '안녕하세요 반갑습니다.'

sp = spm.SentencePieceProcessor()
sp.Load(model)
ret = sp.encode(input, out_type=str)
print(ret)

sp.Load(model2)
ret = sp.encode(input, out_type=str)
print(ret)

['▁안녕하세요', '▁반갑습니다', '.']
['▁', '안', '녕', '하', '세', '요', '▁', '반', '갑', '습', '니', '다', '.']
